In [2]:
import os
import pandas as pd
from glob import glob
import shutil

In [ ]:
# data 폴더 경로 (notebooks 기준으로 상위 ../data)
data_dir = "../../data"

# data 폴더 안의 모든 파일 가져오기
all_files = os.listdir(data_dir)

# csv만 필터링
csv_files = [f for f in all_files if f.endswith(".csv")]

print("CSV 파일 개수:", len(csv_files))
print("CSV 파일 목록:", csv_files)


CSV 파일 개수: 1118
CSV 파일 목록: ['KB국민은행_2025-08-18_2025-08-25.csv', 'KT동마산지점_2025-08-18_2025-08-25.csv', 'KT마산점_2025-08-18_2025-08-25.csv', 'MG새마을금고_2025-08-18_2025-08-25.csv', 'NHF창원대산1단지아파트_2025-08-18_2025-08-25.csv', 'S&T중공업_2025-08-18_2025-08-25.csv', 'SM비즈타운_2025-08-18_2025-08-25.csv', 'STX아파트_2025-08-18_2025-08-25.csv', 'STX칸1차아파트후문_2025-08-18_2025-08-25.csv', 'STX칸2차아파트_2025-08-18_2025-08-25.csv', '㈜동방_2025-08-18_2025-08-25.csv', '가곡마을_2025-08-18_2025-08-25.csv', '가술_2025-08-18_2025-08-25.csv', '가월마을입구_2025-08-18_2025-08-25.csv', '가월윗담_2025-08-18_2025-08-25.csv', '가월입구_2025-08-18_2025-08-25.csv', '가음정본동_2025-08-18_2025-08-25.csv', '가음정사거리_2025-08-18_2025-08-25.csv', '가음정시장_2025-08-18_2025-08-25.csv', '가주동_2025-08-18_2025-08-25.csv', '가주동시내버스회차지_2025-08-18_2025-08-25.csv', '가촌_2025-08-18_2025-08-25.csv', '가촌마을_2025-08-18_2025-08-25.csv', '가포LH1단지_2025-08-18_2025-08-25.csv', '가포고등학교_2025-08-18_2025-08-25.csv', '가포신항_2025-08-18_2025-08-25.csv', '갈전마을_2025-08-18_2025-08-25.csv', '갈전운동장_

In [5]:
df = pd.read_csv("../../data/KB국민은행_2025-08-18_2025-08-25.csv")
df

,정류장명,일자,04(승차),04(하차),05(승차),05(하차),06(승차),06(하차),07(승차),07(하차),...,23(하차),00(승차),00(하차),01(승차),01(하차),02(승차),02(하차),03(승차),03(하차),Unnamed: 51
0,KB국민은행,2025-08-18(월),0,0,8,0,24,0,54,1,...,0,0,0,0,0,0,0,0,0,NaN
1,KB국민은행,2025-08-19(화),0,0,8,0,21,1,53,2,...,0,0,0,0,0,0,0,0,0,NaN
2,KB국민은행,2025-08-20(수),0,0,8,0,18,1,46,1,...,0,0,0,0,0,0,0,0,0,NaN
3,KB국민은행,2025-08-21(목),0,0,11,0,24,1,57,0,...,1,0,0,0,0,0,0,0,0,NaN
4,KB국민은행,2025-08-22(금),0,0,10,0,17,1,51,1,...,0,0,0,0,0,0,0,0,0,NaN
5,KB국민은행,2025-08-23(토),0,0,4,0,18,0,20,1,...,1,0,0,0,0,0,0,0,0,NaN
6,KB국민은행,2025-08-24(일),0,0,3,0,5,0,12,0,...,0,0,0,0,0,0,0,0,0,NaN
7,KB국민은행,2025-08-25(월),0,0,11,0,25,0,53,0,...,0,0,0,0,0,0,0,0,0,NaN


In [8]:
data_dir = "../../data"
cleaned_dir = "../../cleaned"
broken_dir = "../../broken"
incomplete_dir = "../../incomplete"

os.makedirs(cleaned_dir, exist_ok=True)
os.makedirs(broken_dir, exist_ok=True)
os.makedirs(incomplete_dir, exist_ok=True)

csv_files = glob(os.path.join(data_dir, "*.csv"))

# 1️⃣ 마지막 열 삭제 후 cleaned/에 저장
for f in csv_files:
    try:
        df = pd.read_csv(f)
        df = df.iloc[:, :-1]   # 마지막 열 제거
        save_path = os.path.join(cleaned_dir, os.path.basename(f))
        df.to_csv(save_path, index=False)
        print(f"✅ cleaned 저장 완료: {os.path.basename(f)}")
    except Exception as e:
        print(f"❌ 처리 실패: {os.path.basename(f)} → {e}")

# 2️⃣ 인코딩 깨진 파일 찾아 broken/으로 이동
for f in glob(os.path.join(cleaned_dir, "*.csv")):
    try:
        pd.read_csv(f, encoding="utf-8")
    except Exception:
        print(f"🚨 인코딩 깨짐: {os.path.basename(f)} → broken/ 이동")
        shutil.move(f, os.path.join(broken_dir, os.path.basename(f)))

# 3️⃣ 날짜 누락 파일 분류
expected_dates = pd.date_range("2025-08-18", "2025-08-25").strftime("%Y-%m-%d").tolist()

for f in glob(os.path.join(cleaned_dir, "*.csv")):
    try:
        df = pd.read_csv(f)
        dates = df["일자"].str.split("(").str[0].unique().tolist()
        missing = set(expected_dates) - set(dates)

        if missing:
            print(f"⚠️ 날짜 누락: {os.path.basename(f)} → incomplete/ 이동 (누락: {missing})")
            shutil.move(f, os.path.join(incomplete_dir, os.path.basename(f)))
    except Exception as e:
        print(f"❌ 처리 실패: {os.path.basename(f)} → {e}")


✅ cleaned 저장 완료: KB국민은행_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: KT동마산지점_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: KT마산점_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: MG새마을금고_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: NHF창원대산1단지아파트_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: S&T중공업_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: SM비즈타운_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: STX아파트_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: STX칸1차아파트후문_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: STX칸2차아파트_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: ㈜동방_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: 가곡마을_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: 가술_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: 가월마을입구_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: 가월윗담_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: 가월입구_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: 가음정본동_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: 가음정사거리_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: 가음정시장_2025-08-18_2025-08-25.csv
✅ cleaned 저장 완료: 가주동_2025-08-18_2025-08-25.csv
✅ 

In [9]:
df1 = pd.read_csv("../../cleaned/KB국민은행_2025-08-18_2025-08-25.csv")
df1

,정류장명,일자,04(승차),04(하차),05(승차),05(하차),06(승차),06(하차),07(승차),07(하차),...,23(승차),23(하차),00(승차),00(하차),01(승차),01(하차),02(승차),02(하차),03(승차),03(하차)
0,KB국민은행,2025-08-18(월),0,0,8,0,24,0,54,1,...,1,0,0,0,0,0,0,0,0,0
1,KB국민은행,2025-08-19(화),0,0,8,0,21,1,53,2,...,4,0,0,0,0,0,0,0,0,0
2,KB국민은행,2025-08-20(수),0,0,8,0,18,1,46,1,...,0,0,0,0,0,0,0,0,0,0
3,KB국민은행,2025-08-21(목),0,0,11,0,24,1,57,0,...,4,1,0,0,0,0,0,0,0,0
4,KB국민은행,2025-08-22(금),0,0,10,0,17,1,51,1,...,9,0,0,0,0,0,0,0,0,0
5,KB국민은행,2025-08-23(토),0,0,4,0,18,0,20,1,...,5,1,0,0,0,0,0,0,0,0
6,KB국민은행,2025-08-24(일),0,0,3,0,5,0,12,0,...,4,0,0,0,0,0,0,0,0,0
7,KB국민은행,2025-08-25(월),0,0,11,0,25,0,53,0,...,2,0,0,0,0,0,0,0,0,0


In [10]:
CLEANED_DIR = "../../cleaned"
# 출력할 CSV 경로
OUTPUT_PATH = "../../all_dates_merged.csv"

# 모든 CSV 파일 경로 가져오기
csv_files = glob(os.path.join(CLEANED_DIR, "*.csv"))

all_data = []

for file in csv_files:
    try:
        df = pd.read_csv(file, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(file, encoding="cp949")

    all_data.append(df)

# 전체 합치기
merged_df = pd.concat(all_data, ignore_index=True)

# 일자 기준으로 정렬 (문자열 "2025-08-18(월)" → 앞부분 날짜만 사용)
merged_df["일자_정렬"] = merged_df["일자"].str.extract(r"(\d{4}-\d{2}-\d{2})")
merged_df = merged_df.sort_values(["일자_정렬", "정류장명"]).drop(columns=["일자_정렬"])

# 최종 저장
merged_df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
print(f"✅ 모든 날짜 데이터 합쳐서 하나의 CSV 저장 완료: {OUTPUT_PATH}")

C:\Users\TEMP.BOOK-QDV4PVA41A.000\AppData\Local\Temp\ipykernel_26724\3016681598.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df["일자_정렬"] = merged_df["일자"].str.extract(r"(\d{4}-\d{2}-\d{2})")


✅ 모든 날짜 데이터 합쳐서 하나의 CSV 저장 완료: ../../all_dates_merged.csv


In [19]:
import os

#현재 폴더 경로; 작업 폴더 기준
print(os.getcwd())

#현재 파일의 폴더 경로; 작업 파일 기준
# print(os.path.dirname(os.path.realpath(__file__)))

c:\Users\TEMP.BOOK-QDV4PVA41A.000\Desktop\study\project\2025\changwon_drt\code\merge_data


In [21]:
# 1. 기존 통합 CSV 불러오기
merged_path = "../../all_dates_merged.csv"   # 이미 합쳐둔 파일
merged_df = pd.read_csv(merged_path, encoding="utf-8")

# 2. imcomplete 폴더 안 CSV 불러오기
imcomplete_files = glob("../../incomplete/*.csv")

imcomplete_list = []
for f in imcomplete_files:
    try:
        df = pd.read_csv(f, encoding="utf-8")
        imcomplete_list.append(df)
    except UnicodeDecodeError:
        print(f"[WARN] 인코딩 깨짐 → {f}")
    except Exception as e:
        print(f"[ERROR] {f}: {e}")

# 3. imcomplete 파일들을 하나로 합치기
if imcomplete_list:
    imcomplete_df = pd.concat(imcomplete_list, ignore_index=True)
    
    # 일자 정렬을 위해 문자열에서 yyyy-mm-dd만 추출
    imcomplete_df["일자_정렬"] = imcomplete_df["일자"].str.extract(r"(\d{4}-\d{2}-\d{2})")
    merged_df["일자_정렬"] = merged_df["일자"].str.extract(r"(\d{4}-\d{2}-\d{2})")

    # 4. 기존 merged + imcomplete 합치기
    final_df = pd.concat([merged_df, imcomplete_df], ignore_index=True)

    # 5. 정렬 후 임시 컬럼 제거
    final_df = final_df.sort_values(["일자_정렬", "정류장명"]).drop(columns=["일자_정렬"])

    # 6. 최종 저장
    save_path = "../../final_merged.csv"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    final_df.to_csv(save_path, index=False, encoding="utf-8-sig")

    print(f"[INFO] 최종 통합 CSV 저장 완료 → {save_path}")
else:
    print("[INFO] imcomplete 폴더에 합칠 CSV 없음")


C:\Users\TEMP.BOOK-QDV4PVA41A.000\AppData\Local\Temp\ipykernel_26724\3465290611.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  imcomplete_df["일자_정렬"] = imcomplete_df["일자"].str.extract(r"(\d{4}-\d{2}-\d{2})")


[INFO] 최종 통합 CSV 저장 완료 → ../../final_merged.csv


In [23]:
input_path = "../../all_dates_merged.csv"
output_path = "../../all_dates_time_summed.csv"

# 1. CSV 불러오기
df = pd.read_csv(input_path, encoding="utf-8")

# 2. 필요 없는 깨진 열 및 Unnamed 제거
df = df.loc[:, ~df.columns.str.contains("Unnamed")]
df = df.loc[:, ~df.columns.str.contains("Á|À")]

# 3. 필요한 열만 선택 (04~03)
valid_hours = [f"{h:02d}" for h in list(range(4, 24)) + list(range(0, 4))]  # 04~23, 00~03
keep_cols = ["정류장명", "일자"] + [col for col in df.columns if col[:2] in valid_hours]
df = df[keep_cols]

# 4. 시간대별 합치기 (승차 + 하차)
merged_df = df[["정류장명", "일자"]].copy()
for hour in valid_hours:
    ride_col = f"{hour}(승차)"
    drop_col = f"{hour}(하차)"
    if ride_col in df.columns and drop_col in df.columns:
        merged_df[hour] = df[ride_col] + df[drop_col]
    elif ride_col in df.columns:  # 혹시 하차만 없는 경우
        merged_df[hour] = df[ride_col]
    elif drop_col in df.columns:  # 혹시 승차만 없는 경우
        merged_df[hour] = df[drop_col]

# 5. CSV 저장
merged_df.to_csv(output_path, index=False, encoding="utf-8-sig")

In [ ]:
# 1. 좌표 데이터 불러오기
coord_path = "../../경상남도 창원시_버스정류소 위치정보_20241231.csv"   # 업로드한 좌표 데이터 파일명으로 변경
coord_df = pd.read_csv(coord_path, encoding="utf-8")

# 2. 정류소명이 같은 경우 랜덤하게 하나만 남기기
coord_df = coord_df.drop_duplicates(subset=["정류소명"], keep="first")

# 3. 이용자수 데이터 불러오기
usage_path = "../../all_dates_time_summed.csv"   # 기존 정류장별 이용자수 데이터 파일명으로 변경
usage_df = pd.read_csv(usage_path, encoding="utf-8")

# 4. 정류소명 매칭 확인
common_stops = set(usage_df["정류장명"]).intersection(set(coord_df["정류소명"]))
missing_in_coord = set(usage_df["정류장명"]) - set(coord_df["정류소명"])
missing_in_usage = set(coord_df["정류소명"]) - set(usage_df["정류장명"])

print(f"공통 정류장 개수: {len(common_stops)}")
print(f"이용자수에는 있는데 좌표에는 없는 정류장 개수: {len(missing_in_coord)}")
print(f"좌표에는 있는데 이용자수에는 없는 정류장 개수: {len(missing_in_usage)}")

# # 5. 최종 merge (좌표 붙이기)
# final_df = usage_df.merge(coord_df[["정류소명", "X좌표", "Y좌표"]],
#                           left_on="정류장명", right_on="정류소명", how="left")

# # 6. 저장
# final_df.to_csv("all_dates_with_coords.csv", index=False, encoding="utf-8-sig")
# print("최종 통합 파일 저장 완료: all_dates_with_coords.csv")


공통 정류장 개수: 889
이용자수에는 있는데 좌표에는 없는 정류장 개수: 28
좌표에는 있는데 이용자수에는 없는 정류장 개수: 1035


In [10]:
#import pandas as pd
import numpy as np

# 1. 이용자수에는 있는데 좌표에는 없는 정류장
missing_in_coord_clean = [x for x in missing_in_coord if isinstance(x, str) and pd.notna(x)]
missing_in_coord_df = pd.DataFrame(sorted(missing_in_coord_clean), columns=["정류장명"])
missing_in_coord_df["출처"] = "이용자수만 있음"

# 2. 좌표에는 있는데 이용자수에는 없는 정류장
missing_in_usage_clean = [x for x in missing_in_usage if isinstance(x, str) and pd.notna(x)]
missing_in_usage_df = pd.DataFrame(sorted(missing_in_usage_clean), columns=["정류장명"])
missing_in_usage_df["출처"] = "좌표만 있음"

# 3. 두 데이터 합치기
mismatch_df = pd.concat([missing_in_coord_df, missing_in_usage_df], ignore_index=True)

# 4. 확인
print("⚠️ 매칭되지 않은 정류장 리스트:")
display(mismatch_df)

# 5. 저장 (선택)
mismatch_df.to_csv("../../mismatch_stops.csv", index=False, encoding="utf-8-sig")
print("mismatch_stops.csv 파일로 저장 완료")


⚠️ 매칭되지 않은 정류장 리스트:


,정류장명,출처
0,S&T중공업,이용자수만 있음
1,가음정시장.더샵센트럴파크,이용자수만 있음
2,경남대.서중학교,이용자수만 있음
3,경남대학교.농협앞,이용자수만 있음
4,경남대학교남부터미널종점,이용자수만 있음
...,...,...
1057,활천초등학교,좌표만 있음
1058,회성동행정복지센터/합포여중,좌표만 있음
1059,효동마을,좌표만 있음
1060,휴먼시아2단지,좌표만 있음


mismatch_stops.csv 파일로 저장 완료


In [7]:
usage_df

,정류장명,일자,04,05,06,07,08,09,10,11,...,18,19,20,21,22,23,00,01,02,03
0,1366경남센터,2025-08-18(월),0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.15아트센터,2025-08-18(월),0.0,6.0,12.0,29.0,36.0,33.0,20.0,27.0,...,18.0,9.0,14.0,9.0,11.0,1.0,0.0,0.0,0.0,0.0
2,KB국민은행,2025-08-18(월),0.0,8.0,24.0,55.0,42.0,30.0,32.0,36.0,...,33.0,20.0,8.0,27.0,7.0,1.0,0.0,0.0,0.0,0.0
3,KT동마산지점,2025-08-18(월),0.0,8.0,30.0,42.0,64.0,36.0,29.0,27.0,...,47.0,17.0,19.0,15.0,7.0,3.0,0.0,0.0,0.0,NaN
4,KT마산점,2025-08-18(월),0.0,5.0,35.0,100.0,85.0,82.0,80.0,119.0,...,88.0,40.0,24.0,20.0,19.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# 1. 좌표 데이터 불러오기
coord_path = "../../경상남도 창원시_버스정류소 위치정보_20241231.csv"
coord_df = pd.read_csv(coord_path, encoding="utf-8")

# 2. 정류소명 중복 제거 (좌표 데이터에서 같은 이름은 랜덤 1개만 유지)
coord_df = coord_df.drop_duplicates(subset=["정류소명"], keep="first")

# 3. 이용자수 데이터 불러오기
usage_path = "../../all_dates_time_summed.csv"
usage_df = pd.read_csv(usage_path, encoding="utf-8")

# 4. 공통 정류장만 필터링
common_stops = set(usage_df["정류장명"]).intersection(set(coord_df["정류소명"]))
usage_df = usage_df[usage_df["정류장명"].isin(common_stops)]
coord_df = coord_df[coord_df["정류소명"].isin(common_stops)]

# 5. 병합 (좌표 붙이기)
final_df = usage_df.merge(
    coord_df[["정류소명", "X좌표", "Y좌표"]],
    left_on="정류장명", right_on="정류소명", how="left"
)

# 6. 필요 없는 중복 컬럼 제거
final_df = final_df.drop(columns=["정류소명"])

# 7. 저장
final_df.to_csv("../../all_dates_with_coords_filtered.csv", index=False, encoding="utf-8-sig")
print("✅ 최종 통합 완료: all_dates_with_coords_filtered.csv")
print(f"최종 데이터 크기: {final_df.shape}")


✅ 최종 통합 완료: all_dates_with_coords_filtered.csv
최종 데이터 크기: (7128, 28)
